# Workflows

Usually, we want to run more complex pipelines.

The [workflow object](http://nipy.org/nipype/api/generated/nipype.pipeline.engine.html#workflow
)
>Controls the setup and execution of a pipeline of processes.


In [ ]:
import os
try:
    os.chdir(notebook_path)
except:
    notebook_path = os.path.abspath('.')

output_path = os.path.abspath('outputs')
if not os.path.exists(output_path):
    os.mkdir(output_path)
    
wd_path = os.path.join(output_path, '04_workflow')
if not os.path.exists(wd_path):
    os.mkdir(wd_path)
os.chdir(wd_path)
print(wd_path)

In [ ]:
data_path = os.path.join(notebook_path, 'data')
funct_file = os.path.join(data_path, 'ds005_R1.1.0/sub-16/func/sub-16_task-mixedgamblestask_run-01_bold.nii.gz')
print(funct_file)

# A very simple workflow

Let's start with basic workflow definition

In [ ]:
from nipype.pipeline.engine import Node, Workflow
from nipype.interfaces.nipy.preprocess import Trim

workflow_path = os.path.join(wd_path, 'wf_1')
wf = Workflow(name='my_first_wf')
wf.base_dir = workflow_path
wf.config['execution']['crashdump_dir'] = os.path.join(workflow_path, 'crash')
wf.config['execution']['remove_unnecessary_outputs'] = False

Let's define the first node of our wf

In [ ]:
trim = Node(Trim(), 'trim')
trim.inputs.in_file = funct_file
trim.inputs.end_index = 10

If we have only one node in our wf we can add the node to the workflow with __wf.add_nodes()__ and run the entire workflow with __wf.run()__

In [ ]:
wf.add_nodes([trim])
wf.run()

## Now inspect the created directories

In [ ]:
!ls wf_1/my_first_wf/

In [ ]:
!ls wf_1/my_first_wf/trim

# A more complex workflow

Let's start with wf definition

In [ ]:
from nipype.pipeline.engine import Node, Workflow
from nipype.interfaces.nipy.preprocess import Trim, FmriRealign4d

workflow_path = os.path.join(wd_path, 'wf_2')
wf = Workflow(name='my_second_wf')
wf.base_dir = workflow_path
wf.config['execution']['crashdump_dir'] = os.path.join(workflow_path, 'crash')
wf.config['execution']['remove_unnecessary_outputs'] = False

Now we define 2 nodes

In [ ]:
trim = Node(Trim(), 'trim')
trim.inputs.in_file = funct_file
trim.inputs.end_index = 10

realign = Node(FmriRealign4d(), 'realign')
realign.inputs.tr = 3.
realign.inputs.time_interp = True
realign.inputs.slice_order = range(0,34)

We now need to add nodes to our workflow and connect node inputs and outputs.

Instead of using `add_nodes()` we now use __wf.connect()__

Using the following command we can connect the 'out_file' field of trim with the 'in_file' field of realign.

In [ ]:
wf.connect(trim, 'out_file', realign, 'in_file')

and run the entire workflow

In [ ]:
wf.run()

## Now inspect the created directories

In [ ]:
!ls wf_2/my_second_wf

In [ ]:
!ls wf_2/my_second_wf/trim

In [ ]:
!ls wf_2/my_second_wf/realign

# Run workflows

* `wf.run()` runs the workflow on your machine with in one process (core) (equal to `wf.run(plugin='Linear')`)


* `wf.run(plugin='MultiProc', plugin_args={'n_procs' : 2})` runs it on your machine using 2 process in parallel


* `wf.run(plugin='CondorDAGMan')` submits jobs to condor


see http://www.mit.edu/~satra/nipype-nightly/users/plugins.html